## Inferencing Notebook

In [2]:
# import core libaries
import numpy as np
import glob
import os
import sys
import tifffile
from processing_functions import *

# get working directory
path = os.getcwd()
sys.path.append(path)

# import machine learning libraries
import torch
from torchvision import transforms, utils

/Users/jasonfung/miniforge3/envs/ml_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/jasonfung/miniforge3/envs/ml_env/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/jasonfung/miniforge3/envs/ml_env/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowExxb
  Referenced from: /Users/jasonfung/miniforge3/envs/ml_env/lib/python3.9/site-packages/torchvision/image.so
  Expected in: /Users/jasonfung/miniforge3/envs/ml_env/lib/python3.9/site-packages/torch/lib/libc10.dylib
  warn(f"Failed to load image Python extension: {e}")


In [39]:
# directory for models
trained_model = 'ResUNet_62.pth'

# initialize cuda if available
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

# # if using mac
# use_mps = torch.has_mps
# device = torch.device('mps' if use_mps else "cpu")

In [40]:
# define torch unet model
from monai.networks.nets import BasicUNet, UNet

dropout = 0.382
learning_rate = 0.000754
l2 = 0.00421
input_chnl = 1
output_chnl = 4

# model = BasicUNet(spatial_dims=3, 
#                   in_channels = input_chnl,
#                   out_channels = output_chnl,
#                   features = (16, 32, 64, 128, 256, 16),
#                   norm = "batch",
#                   dropout = dropout,
#                )

model = UNet(spatial_dims=3, 
             in_channels = input_chnl,
             out_channels = output_chnl,
             channels = (16, 32, 64, 128, 256),
             strides=(2, 2, 2, 2),
             num_res_units=2,
             norm = "batch",
             dropout = dropout)

model = model.to(device)

In [41]:
model.load_state_dict(torch.load(f'{trained_model}', map_location=torch.device(device)))

<All keys matched successfully>

In [42]:
# processing raw image
lateral_steps = 64
axial_steps = 16
patch_size = (axial_steps, lateral_steps, lateral_steps)
batch_size = 64
# split_size = 0.9
dim_order = (0,4,1,2,3) # define the image and mask dimension order

# Get test images from test images folder
raw_path = f"{path}/Test Images/P_00_200707_N1_15_C2.tif"
raw_img = glob.glob(raw_path)
orig_shape = tifffile.imread(raw_img).shape

# Use patch transform to normalize and transform ndarray(z,y,x) -> tensor(
patch_transform = transforms.Compose([MinMaxScalerVectorized(),
                                      patch_imgs(xy_step = lateral_steps, z_step = axial_steps, patch_size = patch_size, is_mask = False)])


processed_test_img = MyImageDataset(raw_list = raw_img,
                                    mask_list = None,
                                    transform = patch_transform,
                                    device = device,
                                    img_order = dim_order,
                                    mask_order = dim_order,
                                    num_classes = None,
                                    train=False)

In [47]:
processed_test_img

AttributeError: 'MyImageDataset' object has no attribute 'shape'

In [43]:
reconstructed_img = inference(processed_test_img, 
                              model, 
                              batch_size, 
                              patch_size, 
                              orig_shape,
                              shuffle = False)

Inferencing Upper Half of Image


100%|█████████████████████████████████████████████| 2/2 [00:03<00:00,  1.60s/it]


Inferencing Lower Half of Image


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


In [44]:
import napari
viewer = napari.Viewer()
orig_img = tifffile.imread(raw_img)
raw_image = viewer.add_image(orig_img, rgb=False)

In [45]:
label_img = viewer.add_labels(reconstructed_img.astype(int))

/Users/jasonfung/miniforge3/envs/ml_env/lib/python3.9/weakref.py:369: RuntimeWarning: coroutine 'get_msg' was never awaited
  self.data = {}
